In [2]:
import pandas as pd
from pymongo import MongoClient
import random
import datetime
import dotenv
from dotenv import load_dotenv,dotenv_values
import os


dotenv.load_dotenv()
config = dotenv_values(".env")

ATLAS_URI = os.getenv("ATLAS_URI")
CAR_IN_DB = os.getenv("CAR_IN_DB")
CAR_OUT_DB = os.getenv("CAR_OUT_DB")
PARKING_DB = os.getenv("PARKING_DB")

print(ATLAS_URI,CAR_IN_DB,CAR_OUT_DB,PARKING_DB)

localhost:27017 car_in_db car_out_db parking_db


In [22]:
client = MongoClient(ATLAS_URI)
print(client.list_database_names())

['admin', 'car_test', 'config', 'local', 'parking_db', 'test']


In [4]:
# delete all database in MongoDB
client.drop_database(CAR_IN_DB)
client.drop_database(CAR_OUT_DB)
client.drop_database(PARKING_DB)

In [5]:
db = client[PARKING_DB]
parking = db["parking"]
parking.drop()
parking_list = ["A1","A2","A3","A4","B1","B2","B3","B4"]
for i in parking_list:
    parking.insert_one({"stall_id":i,"stall_status":0})

In [6]:
def get_parking_status():
    parking_status = parking.find()
    parking_status = list(parking_status)
    parking_status = pd.DataFrame(parking_status)
    return parking_status

In [ ]:
get_parking_status()

In [10]:
def update_parking(stall_id, stall_status):

    assert stall_status in [0,1], "stall_status must be 0 or 1"
    assert stall_id in parking_list, "stall_id must be A1, A2, A3, A4, B1, B2, B3, B4"

    parking.update_one(
            {"stall_id":stall_id},
            {"$set":{"stall_status":stall_status}}
        )
    return f"{stall_id} updated"

In [11]:
update_parking("A1",1)
get_parking_status()

,_id,stall_id,stall_status
0,642b95641bb02e0611f5be27,A1,1
1,642b95641bb02e0611f5be28,A2,0
2,642b95641bb02e0611f5be29,A3,0
3,642b95641bb02e0611f5be2a,A4,0
4,642b95641bb02e0611f5be2b,B1,0
5,642b95641bb02e0611f5be2c,B2,0
6,642b95641bb02e0611f5be2d,B3,0
7,642b95641bb02e0611f5be2e,B4,0


In [12]:
# def select_partking_method():
def assign_parking():
    parking_status = get_parking_status()
    parking_status = parking_status[parking_status["stall_status"]==0]
    if len(parking_status) == 0:
        return "No parking available"
    else:
        # stall_id = random.choice(parking_status["stall_id"])
        # parking by order of stall_id if there are more than 1 empty stall
        stall_id = parking_status["stall_id"].iloc[0]
        update_parking(stall_id,1)
        return f"Assigned {stall_id}"

In [21]:
assign_parking()
get_parking_status()

,_id,stall_id,stall_status
0,642b95641bb02e0611f5be27,A1,1
1,642b95641bb02e0611f5be28,A2,1
2,642b95641bb02e0611f5be29,A3,1
3,642b95641bb02e0611f5be2a,A4,1
4,642b95641bb02e0611f5be2b,B1,1
5,642b95641bb02e0611f5be2c,B2,1
6,642b95641bb02e0611f5be2d,B3,1
7,642b95641bb02e0611f5be2e,B4,1
